In [0]:
#Utils
import numpy as np
import pandas as pd

#NLP
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import  mark_negation
from nltk.stem import PorterStemmer

In [0]:
#Download resource
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("vader_lexicon")

In [0]:
"""get removed stopword column.
        Parameters
        ----------
        reviews_en : str
            comment in english language
"""
def apply_stopwords(reviews_en):
    tokenize_words_list = []
    for row in reviews_en:
        words = nltk.tokenize.word_tokenize(row)
        tokenize_words = [word.lower() for word in words not in nltk.corpus.stopwords.words("english")]
        tokenize_words_list.append(tokenize_words)
    return tokenize_words_list

In [0]:
"""get stemming words column.
        Parameters
        ----------
        word_list : list of list of string
             list of list of word
"""
def apply_stemming(word_list):
    stemming_list = []
    porter = PorterStemmer()
    for row in word_list:
        stemming_words = [porter.stem(word) for word in word_list]
        stemming_list.append(stemming_words)
    return stemming_list

In [0]:
hospital_list = ["bumrungrad",  "lerdsin", "rajavithi"]

In [0]:
df = pd.read_csv(hospital+"-translated.csv")

In [0]:
rajavithi_df["word_list"] = apply_stopwords(rajavithi_df["reviews_en"])
rajavithi_df.head()


In [0]:
rajavithi_df["stemming_list"] = apply_stemming(rajavithi_df["word_list"])
rajavithi_df.head()